In [13]:
import boto3
from trp import Document
import cv2

from googletrans import Translator, constants
translator = Translator()

In [14]:
card_identification = {'republic of croatia' : {'country':'Croatia',
                                            'Types_of_card':['identity card','driving licence']},
                    'belgium' : {'country':'Belgium',
                                            'Types_of_card':['identity card','driving licence']}
                                            
                                            }

In [22]:
# S3 Bucket Data
def extract_top(file_path,s3BucketName):
    # s3BucketName = "kshatra-ai-jaydev"
    # file_path = "./id_card_imgs/croatia3.jpeg"
    img = cv2.imread(file_path)
    img = cv2.resize(img, (1260, 820))
    list = file_path.split('/')
    cv2.imwrite(file_path, img)
    file_name = list[-1]


    s3 = boto3.client('s3')
    s3.upload_file(file_path ,s3BucketName, file_name)
    print(".....DONE -UPLOADING.......")
    FormdocumentName = file_name
    PlaindocumentName = file_name
    # Amazon Textract client
    textractmodule = boto3.client('textract')
    top_list = []
    #1. PLAINTEXT detection from documents:
    response = textractmodule.detect_document_text(Document={'S3Object': {'Bucket': s3BucketName,'Name': PlaindocumentName}})

    print ('------------- Print Plaintext detected text ------------------------------')
    i = 0
    for item in response["Blocks"]:
        if item["BlockType"] == "LINE":
            i = i+1
            if i < 10:
                #print ('\033[92m'+str(item['Text'])+'\033[92m')
                key = translator.translate(str(item['Text']))
                ket_txt = key.text.lower()
                top_list.append(ket_txt)
                print("abc")           
    return top_list

In [25]:
def identify_country(card_identification,file_path,s3BucketName):
    top_list = extract_top(file_path,s3BucketName)
    for a in range (len(top_list)):
        if top_list[a] in card_identification:
            cntry =  card_identification[top_list[a]]['country']
            types_of_card =  card_identification[top_list[a]]['Types_of_card']
            # print("Country is", cntry)
            break         
        else:
            print("Country does not exist in dict.")
    return cntry, types_of_card, top_list



def identify_card(card_identification, file_path,s3BucketName):
    country, card_types, top_list= identify_country(card_identification, file_path, s3BucketName)

    for b in range (len(top_list)):
        if top_list[b] in card_types:
            card =  top_list[b]
            return country,card 
            break       
        
    print("card type does not exist in dict.")

In [26]:
identify_card(card_identification, file_path = "./id_card_imgs/croatia3.jpeg", s3BucketName = "kshatra-ai-jaydev")

.....DONE -UPLOADING.......
------------- Print Plaintext detected text ------------------------------
abc
abc
abc
abc
abc
abc
abc
abc
abc


('Croatia', 'identity card')